In [ ]:
# !pip install tensorflow==2.3.1 nltk==3.5 colorama==0.4.3 numpy==1.18.5 scikit_learn==0.23.2 Flask==1.1.2

## Imports 

In [ ]:
import json 
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
import sys
import os

In [ ]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

## Reading Intent File

In [ ]:
with open('Intent.json',encoding="utf-8") as file:
    intent_data_json = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []

response_json = {}
for intent in intent_data_json['intents']:
    for pattern in intent['text']:
        training_sentences.append(pattern)
        training_labels.append(intent['intent'])

    response_json[intent['intent']] = intent['responses']
    
    responses.append(intent['responses'])
    
    if intent['intent'] not in labels:
        labels.append(intent['intent'])

with open('Intent_response.json','w',encoding="utf-8") as fp:
    json.dump(response_json, fp,indent=4)
    

In [ ]:
intent_data_pd = pd.json_normalize(intent_data_json, record_path =['intents'])
intent_data_pd["text"].iloc[0].append("hi")


## Label Encoder

In [ ]:
def train(training_labels, training_sentences):
    num_classes = len(np.unique(training_labels))
    lbl_encoder = LabelEncoder()
    lbl_encoder.fit(training_labels)
    training_labels = lbl_encoder.transform(training_labels)
    vocab_size = 1000
    embedding_dim = 16
    max_len = 20
    oov_token = "<OOV>"

    tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
    tokenizer.fit_on_texts(training_sentences)
    word_index = tokenizer.word_index
    sequences = tokenizer.texts_to_sequences(training_sentences)
    padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(16, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy', 
                  optimizer='adam', metrics=['accuracy'])

    model.summary()
    epochs = 500
    
#     old_stdout = sys.stdout # backup current stdout
#     sys.stdout = open(os.devnull, "w")
    history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs, verbose='1')
#     sys.stdout = old_stdout # reset old stdout

    model.save("chat_model")

    import pickle

    # to save the fitted tokenizer
    with open('tokenizer.pickle', 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

    # to save the fitted label encoder
    with open('label_encoder.pickle', 'wb') as ecn_file:
        pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
%%capture
train(training_labels, training_sentences)

In [ ]:
def set_user_name(input_query):
    global user_data
    user_data["User"]["Name"] = input_query.split()[-1]
    return None
def get_user_name():
    global user_data
    return "Your name is " + user_data["User"]["Name"] if user_data["User"]["Name"] else "Can you tell me your name?"
def greet_by_name():
    global user_data
    return "Nice to Meet you " + user_data["User"]["Name"]
def do_nothing():
    pass
def learn_new_question():
    global training_sentences
    global training_labels
    global response_json
    global intent_data_pd
    print(Fore.GREEN + "ChatBot:" + str("[learnFunc]") + Style.RESET_ALL , "Can you tell me the question?")
    print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
    sentence = input()
    training_sentences.append(sentence)
    print(Fore.GREEN + "ChatBot:" + str("[learnFunc]") + Style.RESET_ALL , "Great. What is the answer?")
    print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
    answer = input()
    print(Fore.GREEN + "ChatBot:" + str("[learnFunc]") + Style.RESET_ALL , "Last Question. What is the category?")
    print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
    label = input()
    training_labels.append(label)
    response_json[label] = response_json.get(label,[])
    response_json[label].append(answer)
    print(Fore.GREEN + "ChatBot:" + str("[learnFunc]") + Style.RESET_ALL , "Thanks. I will update my database soon!")
    
    if not intent_data_pd[intent_data_pd.intent == label].empty:
        intent_data_pd[intent_data_pd.intent == label]["text"].iloc[0].append(sentence)
        intent_data_pd[intent_data_pd.intent == label]["responses"].iloc[0].append(answer)
    else:
        new_df = pd.DataFrame([[label,[sentence],[answer]]],columns=["intent","text","responses"])
        frames = [intent_data_pd,new_df]
        intent_data_pd = pd.concat(frames,ignore_index=True)
    store_current_data()
def store_current_data():
    global intent_data_pd
    with open("Intent.json","w") as output_file:
        new_json = {}
        new_json["intents"] = json.loads(intent_data_pd.to_json(orient="records"))
        json.dump(new_json, output_file, indent=4)

In [ ]:
%%capture
def update_model():
    global training_sentences
    global training_labels
    global response_json
    train(training_sentences, training_labels)
    load_model()

In [ ]:
import json 
import numpy as np
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder

import colorama 
colorama.init()
from colorama import Fore, Style, Back

import random
import pickle

with open("Intent_response.json",encoding="utf-8") as intent_file:
    data = json.load(intent_file)
with open("User.json", encoding="utf-8") as user_file:
    user_data = json.load(user_file)

def load_model():
    # load trained model
    model = keras.models.load_model('chat_model')

    # load tokenizer object
    with open('tokenizer.pickle', 'rb') as handle:
        tokenizer = pickle.load(handle)

    # load label encoder object
    with open('label_encoder.pickle', 'rb') as enc:
        lbl_encoder = pickle.load(enc)
    return model,tokenizer,lbl_encoder

def chat():   
    model,tokenizer,lbl_encoder = load_model()
    # parameters
    max_len = 20
    update_response = False
    last_query = ''
    
    iters = 100
    while iters:
        print(Fore.LIGHTBLUE_EX + "User: " + Style.RESET_ALL, end="")
        inp = input()
        if inp.lower() == "quit" or inp.lower() == "exit" or inp.lower() == "bye":
            break

        result = model.predict(keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inp]),truncating='post', maxlen=max_len))
        tag = lbl_encoder.inverse_transform([np.argmax(result)])
#             print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,tag)
        iters = iters - 1
        for key,val in response_json.items():
#                 if tag == 'feedback':
#                     update_response = True
            if key == tag:
                if "Func" in str(tag):
                    resp_func = eval(val[0])
                    _,resp = resp_func(inp)
                else:
                    resp = np.random.choice(val).replace("<HUMAN>",user_data["User"]["Name"])

                print(Fore.GREEN + "ChatBot:" + str(tag) + Style.RESET_ALL , resp)
                last_query = inp
                break

#         print(Fore.GREEN + "ChatBot:" + Style.RESET_ALL,random.choice(responses))

print(Fore.YELLOW + "Start messaging with the bot (type quit to stop)!" + Style.RESET_ALL)


In [ ]:
chat()

In [156]:
store_current_data()

In [154]:
intent_data_pd

,intent,text,responses
0,Greeting,"[Hi, Hi there, Hola, Hello, Hello there, Hya, ...","[Hi!, Hello!, Hola!,]"
1,GreetingResponseFunc,"[I am called , My name is , My user is , This ...",[lambda input_query: (set_user_name(input_quer...
2,CourtesyGreeting,"[How are you?, Hi how are you?, Hello how are ...","[Hello, I am great, how are you?, Hello, how a..."
3,CourtesyGreetingResponse,"[Good thanks!, I am good., Doing good, Great, ...","[Great! How can I help?, Good! how can I help ..."
4,CurrentHumanQueryFunc,"[What is my name?, What do you call me?, Who d...","[lambda input_query: (do_nothing(),get_user_na..."
5,NameQuery,"[What is your name?, What could I call you?, W...","[You can call me Geni, You may call me Geni, C..."
6,NeedHelp,"[I need some help, Help me, I require assistan...","[How can I help?, how can I help you?, What ca..."
7,TimeQueryFunction,"[What is the time?, What's the time?, Do you k...","[lambda x, y : "" I don't know the time "" ]"
8,Thanks,"[OK thank you, OK thanks, OK, Thanks, Thank yo...","[No problem!, Happy to help!, Any time!, My pl..."
9,NotTalking2U,"[I am not talking to you, I was not talking to...","[OK, No problem, Right]"
